In [1]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact
from matplotlib.gridspec import GridSpec
# from matplotlib.colors import LinearSegmentedColormap
plt.style.use('./deeplearning.mplstyle')

In [3]:
# dlblue = '#0096ff'; dlorange = '#FF9300'; dldarkred='#C00000'; dlmagenta='#FF40FF'; dlpurple='#7030A0';
# dlcolors = [dlblue, dlorange, dldarkred, dlmagenta, dlpurple]
# dlc = dict(dlblue = '#0096ff', dlorange = '#FF9300', dldarkred='#C00000', dlmagenta='#FF40FF', dlpurple='#7030A0')
# n_bin = 5
# dlcm = LinearSegmentedColormap.from_list(
#         'dl_map', dlcolors, N=n_bin)

In [2]:
x_train = np.array([1.0, 1.7, 2.0, 2.5, 3.0, 3.2])
y_train = np.array([250, 300, 480,  430,   630, 730,])

In [8]:
def compute_cost(X, Y, w, b):
    return np.mean((np.dot(X, w) + b -Y)**2)/2

In [3]:
def plt_house(X, Y, f_wb=None, ax=None):
    if not ax:
        fig, ax = plt.subplots(1,1)
    ax.scatter(X, Y, marker='x', c='r', label="Actual Value")
    if f_wb is not None:
        ax.plot(X, f_wb, c='b', label="Our Prediction")
    ax.set_title("Housing Prices")
    ax.set_ylabel("Price (in 1000s of dollars)")
    ax.set_xlabel("Size (1000 sqft)")

In [4]:
def mk_cost_lines(x, y, w, b, ax):
    cstr = "cost = (1/m)*("
    label = 'cost for point'
    ctot = 0
    addedbreak = False
    for p in zip(x, y):
        f_wb_p = p[0] * w + b
        c_p = ((f_wb_p - p[1])**2)/2
        c_p_txt = f'{c_p:0.0f}'
        ax.vlines(p[0], p[1], f_wb_p, lw=3, color='purple', ls='dotted', label=label)
        label = None
        cxy = [p[0], p[1] + (f_wb_p - p[1])/2]
        ax.annotate(c_p_txt, xy=cxy, xycoords= 'data', fontsize=8, c='purple', 
                    xytext=(5,0), textcoords="offset points")
        cstr += f'{c_p_txt} +'
        ctot += c_p
        if len(cstr) >= 35 and not addedbreak:
            cstr += '\n'
            addedbreak = True
    cost = ctot/len(x)
    cstr = cstr[:-1] + f') = {cost:0.0f}'
    ax.text(0.15, 0.02, cstr, fontsize=8, transform=ax.transAxes, c='purple')

In [5]:
class SliderState:
    """Stores and manages dynamic artists"""
    def __init__(self):
        self.dyn_items = []

In [6]:
def plt_intuition(X, Y):
    w_range = np.array([209-200, 209+200])
    w_array = np.arange(*w_range, 5)
    c_array = np.zeros_like(w_array)
    fixed_b = 2.4

    for i in range(len(w_array)):
        c_array[i] = compute_cost(X, Y, w_array[i], fixed_b)


    plt.close('all')
    fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(8,4))
    fig.canvas.toolbar_position = 'bottom'

    ax[1].set_xlim(w_range[0]-20, w_range[-1]+20)
    ax[1].set_ylim(0, np.max(c_array)*1.1)
    ax[1].plot(w_array, c_array)
    ax[1].set_title(f"Cost vs w, (b fixed at {fixed_b})")
    ax[1].set_ylabel("Cost")
    ax[1].set_xlabel("w")

    state = SliderState()

    @interact(w=(*w_range, 10), continuous_update=False)
    def update(w=149):
        ax[0].clear()
        plt_house(X, Y, np.dot(X, w) + fixed_b, ax[0])
        mk_cost_lines(X, Y, w, fixed_b, ax[0])
        ax[0].legend()

        cur_cost = compute_cost(X, Y, w, fixed_b)

        for artist in state.dyn_items:
            try:
                artist.remove()
            except Exception as e:
                pass
        state.dyn_items = []

        scatter = ax[1].scatter(w, cur_cost, s=100, zorder=10, c='r', label=f'cost at w={w}')
        hlines = ax[1].hlines(cur_cost, ax[1].get_xlim()[0], w, lw=4, ls='dotted', color='purple')
        vlines = ax[1].vlines(w, ax[1].get_ylim()[0], cur_cost, lw=4, ls='dotted', color='purple')
        legend = ax[1].legend(loc='upper center')

        state.dyn_items.extend([scatter, hlines, vlines, legend])

        fig.suptitle(f"Minimize Cose: Cost = {cur_cost:0.0f}", fontsize=10)
        fig.canvas.draw()

In [ ]:
plt_intuition(x_train, y_train)

In [10]:
def plt_stationary(X, Y):
    fig = plt.figure(figsize=(9,8))
    fig.set_facecolor('#ffffff')
    fig.canvas.toolbar_position = 'top'
    gs = GridSpec(2,2, figure=fig)
    ax0 = fig.add_subplot(gs[0, 0])
    ax1 = fig.add_subplot(gs[0,1])
    ax2 = fig.add_subplot(gs[1, :], projection='3d')
    ax = np.array([ax0, ax1, ax2])

    w_range = np.array([200-300, 200+300])
    b_range = np.array([50-300, 50+300])
    w_space = np.linspace(*w_range, 100)
    b_space = np.linspace(*b_range, 100)
    
    tmp_b, tmp_w = np.meshgrid(b_space, w_space)
    z = np.zeros_like(tmp_b)
    for i in range(tmp_w.shape[0]):
        for j in range(tmp_w.shape[1]):
            z[i][j] = compute_cost(X, Y, tmp_w[i][j], tmp_b[i][j])
            if z[i][j] == 0:
                z[i][j] = 1e-6

    w0, b0 = 200, -100
    f_wb_0=np.dot(X, w0) + b0
    cst = compute_cost(X, Y, w0, b0)
    plt_house(X, Y, f_wb=f_wb_0, ax=ax[0])
    mk_cost_lines(X, Y, w0, b0, ax=ax[0])
    ax[0].legend()

    CS = ax[1].contour(tmp_w, tmp_b, np.log(z), levels=12, linewidths=2, alpha=0.7)
    ax[1].set_title('Cost(w,b)')
    ax[1].set_xlabel('w', fontsize=10)
    ax[1].set_ylabel('b', fontsize=10)
    ax[1].set_xlim(*w_range)
    ax[1].set_ylim(*b_range)
    cscat = ax[1].scatter(w0, b0, s=100, zorder=10, color='purple', label='cost for current w and b')
    chline = ax[1].hlines(b0,ax[1].get_xlim()[0],w0,linewidth=4,ls='dotted',color='purple')
    cvline = ax[1].vlines(w0,ax[1].get_ylim()[0],b0,linewidth=4,ls='dotted',color='purple')
    cannotate = ax[1].annotate(f'Cost={cst:.0f}, w={w0:.0f}, b={b0:.0f}', xy=(w0, b0), xytext=(4,4), textcoords='offset points', bbox=dict(facecolor='white'), size=8)
    ax[1].text(0.5, 0.95, 'Click to choose w, b',bbox=dict(facecolor='white', ec='black'), fontsize=8,    transform=ax[1].transAxes,horizontalalignment='center', verticalalignment='center')

    ax[2].plot_surface(tmp_w, tmp_b, z, cmap = "Spectral_r", alpha=0.7, antialiased=True)
    ax[2].plot_wireframe(tmp_w, tmp_b, z, color='k', alpha=0.1)
    ax[2].set_xlabel('$w$')
    ax[2].set_ylabel('$b$')
    ax[2].zaxis.set_rotate_label(False)
    ax[2].set_title('Cost(w,b) \n You can rotate this figure', size=12)
    ax[2].xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax[2].yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax[2].zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax[2].set_zlabel('J(w,b)\n\n', rotation=90)
    sscat= ax[2].scatter(w0, b0, cst, marker='x', s=100)
    ax[2].view_init(30, -120)

    return fig, ax, [cscat, chline, cvline, cannotate, sscat]

In [11]:
class plt_update_onclick:
    def __init__(self, fig, ax, x, y, dyn_items):
        self.fig = fig
        self.ax = ax
        self.x = x
        self.y = y
        self.dyn_items = dyn_items
        self.cid = self.fig.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        if event.inaxes == self.ax[1]:
            ws, bs = event.xdata, event.ydata
            if ws is None or bs is None:
                return
            xlim, ylim = self.ax[1].get_xlim(), self.ax[1].get_ylim()
            if not (xlim[0] <= ws <= xlim[1] and ylim[0] <= bs <= ylim[1]):
                return
            cst = compute_cost(self.x, self.y, ws, bs)

            self.ax[0].clear()
            f_wb = np.dot(self.x, ws) + bs
            plt_house(self.x, self.y, f_wb=f_wb, ax=self.ax[0])
            mk_cost_lines(self.x, self.y, ws, bs, ax=self.ax[0])
            self.ax[0].legend()

            for artist in self.dyn_items:
                artist.remove()
            self.dyn_items.clear()

            a = self.ax[1].scatter(ws, bs, s=100, zorder=10, color='purple', label=f'cost = {cst:.2f} for w = {ws:.2f}, b = {bs:.2f}')
            b = self.ax[1].hlines(bs, xlim[0], ws, lw=4, ls='dotted', color='purple')
            c = self.ax[1].vlines(ws, ylim[0], bs, lw=4, ls='dotted', color='purple')
            d = self.ax[1].annotate(f'Cost={cst:.0f}, w={ws:.0f}, b={bs:.0f}', xy=(ws, bs), xytext=(4,4), textcoords='offset points', bbox=dict(facecolor='white'), size=8)
            e = self.ax[2].scatter(ws, bs, cst, marker='x', s=100)

            self.dyn_items = [a,b,c,d,e]
            self.fig.canvas.draw()

In [ ]:
plt.close('all')
fig, ax, dyn_items = plt_stationary(x_train, y_train)
updater = plt_update_onclick(fig, ax, x_train, y_train, dyn_items)

In [13]:
def soup_bowl():
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_rotate_label(False)
    ax.view_init(45, -120)

    wb_range = np.array([-20, 20])
    w_array, b_array = np.linspace(*wb_range, 100), np.linspace(*wb_range, 100)

    b, w = np.meshgrid(b_array, w_array)
    z = np.zeros_like(b)
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            # z[i][j] = w[i][j] ** 2 + b[i][j]
            z[i][j] = compute_cost(np.array([1,2,3,4,5]), np.array([4,6,8,10,12]), w[i][j], b[i][j])

    ax.plot_surface(w, b, z, cmap = "Spectral_r", alpha=0.7, antialiased=False)
    ax.plot_wireframe(w, b, z, color='k', alpha=0.1)
    ax.set_xlabel('$w$')
    ax.set_ylabel('$b$')
    ax.set_zlabel('$J(w,b)$', rotation=90)
    ax.set_title("$J(w,b)$\n [You can rotate this figure]", size=15)
    plt.show()

In [ ]:
fig.clear('all')
soup_bowl()